Vehicle Insurance Claims Prediction

The "Vehicle Insurance Claims Prediction" project was part of the "Allstate Claim Prediction Challenge" on Kaggle, held on 13-07-2011, with a focus on predicting claims payments. The goal was to develop predictive models to estimate insurance claims payments based on historical data and relevant features, using various machine learning techniques, data preprocessing methods, and feature engineering approaches to improve prediction accuracy.

Import necessary libraries:

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

Import data:

In [46]:
train_data = pd.read_csv(r'D:\ML_Projects\Vehicle_Insurance_Claims_Prediction\train_set.csv')
y_train = train_data['Claim_Amount']

test_data = pd.read_csv(r'D:\ML_Projects\Vehicle_Insurance_Claims_Prediction\test_set.csv')
example_entry = pd.read_csv(r'D:\ML_Projects\Vehicle_Insurance_Claims_Prediction\example_entry.csv')
test_data = test_data.merge(example_entry, on='Row_ID', how='left')
y_test = test_data['Claim_Amount']

In [60]:
print('Number of training samples: ', format(len(train_data),','))
print('Number of test samples: ', format(len(test_data), ','))
print('Train % : ', round(len(train_data) / (len(train_data) + len(test_data)), 3)*100,'%')
print('Test %  : ', round(len(test_data) / (len(train_data) + len(test_data)), 3)*100, '%')

Number of training samples:  13,184,290
Number of test samples:  4,314,865
Train % :  75.3 %
Test %  :  24.7 %


In [63]:
print('Features in training data: \n', train_data.columns)
print('Features in test data: \n', test_data.columns)

Features in training data: 
 Index(['Row_ID', 'Household_ID', 'Vehicle', 'Calendar_Year', 'Model_Year',
       'Blind_Make', 'Blind_Model', 'Blind_Submodel', 'Cat1', 'Cat2', 'Cat3',
       'Cat4', 'Cat5', 'Cat6', 'Cat7', 'Cat8', 'Cat9', 'Cat10', 'Cat11',
       'Cat12', 'OrdCat', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6',
       'Var7', 'Var8', 'NVCat', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4',
       'Claim_Amount'],
      dtype='object')
Features in test data: 
 Index(['Row_ID', 'Household_ID', 'Vehicle', 'Calendar_Year', 'Model_Year',
       'Blind_Make', 'Blind_Model', 'Blind_Submodel', 'Cat1', 'Cat2', 'Cat3',
       'Cat4', 'Cat5', 'Cat6', 'Cat7', 'Cat8', 'Cat9', 'Cat10', 'Cat11',
       'Cat12', 'OrdCat', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6',
       'Var7', 'Var8', 'NVCat', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4',
       'Claim_Amount'],
      dtype='object')


In [37]:
train_data.head()

,Household_ID,Vehicle,Calendar_Year,Model_Year,Blind_Make,Blind_Model,Blind_Submodel,Cat1,Cat2,Cat3,...,Var4,Var5,Var6,Var7,Var8,NVCat,NVVar1,NVVar2,NVVar3,NVVar4
0,1,3,2005,2005,K,K.78,K.78.2,D,C,F,...,0.908351,1.008912,0.261040,0.907793,-0.077998,M,-0.23153,-0.266117,-0.272337,-0.251419
1,2,2,2005,2003,Q,Q.22,Q.22.3,B,C,A,...,0.485509,1.240851,0.432987,-0.726459,0.204785,O,-0.23153,-0.266117,-0.272337,-0.251419
2,3,1,2005,1998,AR,AR.41,AR.41.1,B,?,A,...,-1.679445,-0.971487,-1.405797,-0.837048,-1.176858,F,-0.23153,-0.266117,-0.272337,-0.251419
3,3,1,2006,1998,AR,AR.41,AR.41.1,B,?,A,...,-1.679445,-0.971487,-1.405797,-0.837048,-1.176858,F,-0.23153,-0.266117,-0.272337,-0.251419
4,3,2,2005,2001,D,D.20,D.20.0,J,C,B,...,1.838605,0.812656,2.112691,1.534462,2.347260,F,-0.23153,-0.266117,-0.272337,-0.251419


In [66]:
train_data.describe()

,Row_ID,Household_ID,Vehicle,Calendar_Year,Model_Year,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Amount
count,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07
mean,6.592146e+06,4.128242e+06,1.894002e+00,2.006052e+03,1.999312e+03,-1.011925e-02,-6.508702e-02,-2.543391e-02,-5.456792e-02,3.838594e-03,-4.012271e-02,-2.421288e-02,-5.856059e-02,1.468410e-02,1.751169e-02,1.354226e-02,1.851376e-02,1.360658e+00
std,3.805977e+06,2.248415e+06,1.173861e+00,8.123720e-01,5.211866e+00,9.800609e-01,9.684165e-01,1.018902e+00,9.680170e-01,9.910490e-01,9.792078e-01,1.006433e+00,1.003954e+00,1.031040e+00,1.038212e+00,1.027748e+00,1.034274e+00,3.900103e+01
min,1.000000e+00,1.000000e+00,1.000000e+00,2.005000e+03,1.981000e+03,-2.578222e+00,-2.493393e+00,-2.790335e+00,-2.508216e+00,-3.350344e+00,-2.376657e+00,-2.778491e+00,-2.163042e+00,-2.315299e-01,-2.661168e-01,-2.723372e-01,-2.514189e-01,0.000000e+00
25%,3.296073e+06,2.184932e+06,1.000000e+00,2.005000e+03,1.996000e+03,-6.658971e-01,-8.161519e-01,-8.696874e-01,-7.830189e-01,-6.860239e-01,-6.887650e-01,-8.984857e-01,-6.517680e-01,-2.315299e-01,-2.661168e-01,-2.723372e-01,-2.514189e-01,0.000000e+00
50%,6.592146e+06,4.257083e+06,2.000000e+00,2.006000e+03,2.000000e+03,-3.123581e-01,-1.245062e-01,-2.217581e-01,-1.064709e-01,-1.150981e-01,-2.372568e-01,-4.684193e-01,-2.568567e-01,-2.315299e-01,-2.661168e-01,-2.723372e-01,-2.514189e-01,0.000000e+00
75%,9.888218e+06,6.281433e+06,2.000000e+00,2.007000e+03,2.003000e+03,4.429298e-01,4.806838e-01,7.269956e-01,4.855086e-01,5.331405e-01,4.973212e-01,8.217801e-01,3.409799e-01,-2.315299e-01,-2.661168e-01,-2.723372e-01,-2.514189e-01,0.000000e+00
max,1.318429e+07,7.542113e+06,2.900000e+01,2.007000e+03,2.009000e+03,5.143392e+00,7.829420e+00,5.563325e+00,7.589263e+00,4.018167e+00,4.584289e+00,4.127148e+00,4.735074e+01,6.627110e+00,8.883081e+00,8.691144e+00,6.388802e+00,1.144075e+04
